In [1]:
import pandas as pd
import numpy as np
import re
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
from db import get_pd_from_table
from konlpy.tag import Mecab
from tqdm import tqdm

In [2]:
books = get_pd_from_table('books')
books.drop(['isbn', 'pubDate', 'img', 'rate', 'bestseller', 'similarity'], axis=1, inplace=True)
books.dropna(axis=0, inplace=True)
books.head()

,id,title,author,publisher,summary,genre
0,1,리스트 고독한가운데 신의 축복 (984),마스트미디어 편집부 (지은이),마스트미디어,,예술/대중문화
1,2,거짓말이다 + 꽃다발 상품권 세트,김탁환 (지은이),북스피어,김탁환 작가의 &lt;거짓말이다&gt;와 꽃다발을 함께 구입할 수 있는 기획 상품....,소설/시/희곡
2,3,[헌금봉투] 주일헌금 201 (1속50매) - VJ-1311-201,비전북 편집부 (지은이),비전북(VisionBook),비전아트 고급캘리헌금봉투는 캘리그래피를 담은 우아한 디자인을 더했다. 두꺼운 재질의...,종교/역학
3,4,[헌금봉투] 주일헌금 202 (1속50매) - VJ-1311-202,비전북 편집부 (지은이),비전북(VisionBook),비전아트 고급캘리헌금봉투는 캘리그래피를 담은 우아한 디자인을 더했다. 두꺼운 재질의...,종교/역학
4,5,엘르 Elle E형 2022.8 (표지 : NCT 재현) (부록없음) - 주요기사 ...,허스트중앙 편집부 (지은이),허스트중앙(Hearst-Joongang),,잡지


In [3]:
summaries = []
for book in books.values:
    summary = book[1]
    summary += book[2]
    summary += book[3]
    summary += book[4]
    summary += book[5]
    summaries.append(summary)
books['summary'] = summaries
books.drop(['title','author','publisher','genre'], axis=1, inplace=True)
books.head()

,id,summary
0,1,리스트 고독한가운데 신의 축복 (984)마스트미디어 편집부 (지은이)마스트미디어예술...
1,2,거짓말이다 + 꽃다발 상품권 세트김탁환 (지은이)북스피어김탁환 작가의 &lt;거짓말...
2,3,[헌금봉투] 주일헌금 201 (1속50매) - VJ-1311-201비전북 편집부 (...
3,4,[헌금봉투] 주일헌금 202 (1속50매) - VJ-1311-202비전북 편집부 (...
4,5,엘르 Elle E형 2022.8 (표지 : NCT 재현) (부록없음) - 주요기사 ...


In [4]:
# 불용어 가져오기
stopwords = []
with open("data/hangul_stopword.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()  # 줄 끝의 줄 바꿈 문자를 제거한다.
        line = line.replace('\n', '')
        stopwords.append(line)
print(stopwords)

['이', '있', '하', '것', '들', '그', '되', '수', '이', '보', '않', '없', '나', '사람', '주', '아니', '등', '같', '우리', '때', '년', '가', '한', '지', '대하', '오', '말', '일', '그렇', '위하', '때문', '그것', '두', '말하', '알', '그러나', '받', '못하', '일', '그런', '또', '문제', '더', '사회', '많', '그리고', '좋', '크', '따르', '중', '나오', '가지', '씨', '시키', '만들', '지금', '생각하', '그러', '속', '하나', '집', '살', '모르', '적', '월', '데', '자신', '안', '어떤', '내', '내', '경우', '명', '생각', '시간', '그녀', '다시', '이런', '앞', '보이', '번', '나', '다른', '어떻', '여자', '개', '전', '들', '사실', '이렇', '점', '싶', '말', '정도', '좀', '원', '잘', '통하', '소리', '놓']


In [5]:
# summary 전처리
books['summary'] = books['summary'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
mecab = Mecab()
corpus_list = []
for index, row in tqdm(books.iterrows(), total=len(books)):
    text = row['summary']
    tokenized_sentence = mecab.morphs(text)
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    corpus_list.append(stopwords_removed_sentence)
books['summary'] = corpus_list
books.head()

/var/folders/9d/4103_0g54_s9yr74zb4kcm340000gn/T/ipykernel_94734/3166800422.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  books['summary'] = books['summary'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
100%|██████████| 76118/76118 [00:17<00:00, 4230.11it/s]


,id,summary
0,1,"[리스트, 고독, 가운데, 신, 의, 축복, 마스트, 미디어, 편집부, 지은, 마스..."
1,2,"[거짓말, 다, 꽃다발, 상품권, 세트, 김탁환, 지은, 북, 스피어, 김탁환, 작..."
2,3,"[헌, 금봉, 투, 주일, 헌금, 매, 비, 전북, 편집부, 지은, 비, 전북, 비..."
3,4,"[헌, 금봉, 투, 주일, 헌금, 매, 비, 전북, 편집부, 지은, 비, 전북, 비..."
4,5,"[엘르, 형, 표지, 재현, 부록, 음, 주요, 기사, 재현, 김, 태리, 아이, ..."


In [6]:
word2vec_model = Word2Vec.load('data/ko.bin')
word2vec_model.wv.save_word2vec_format('data/ko.bin.gz', binary=True)
a = word2vec_model.wv.most_similar("계발")
a

[('고취', 0.6351145505905151),
 ('증진', 0.6197105050086975),
 ('교화', 0.6087900400161743),
 ('제고', 0.6057854890823364),
 ('실현', 0.6031328439712524),
 ('극대화', 0.5928179621696472),
 ('단련', 0.5917676687240601),
 ('향상', 0.591032862663269),
 ('창의력', 0.5878887176513672),
 ('실천', 0.5872039794921875)]

In [7]:
word2vec_model = Word2Vec(size = 200, window=5, min_count = 2, workers = -1)
word2vec_model.build_vocab(corpus_list)
word2vec_model.intersect_word2vec_format('data/ko.bin.gz', lockf=1.0, binary=True)
word2vec_model.train(corpus_list, total_examples = word2vec_model.corpus_count, epochs = 15)

(0, 0)

In [17]:
a = word2vec_model.wv.most_similar("태리")
a

[('정명오', 0.2842499613761902),
 ('시놉시스', 0.2808126211166382),
 ('이순일', 0.27146995067596436),
 ('시킴', 0.2571180462837219),
 ('플래닝', 0.2547532320022583),
 ('숑', 0.2509073317050934),
 ('카사이', 0.24893814325332642),
 ('히다', 0.24527166783809662),
 ('아카사카', 0.24466341733932495),
 ('공공장소', 0.24441102147102356)]

In [14]:
def get_document_vectors(document_list):
    document_embedding_list = []

    # 각 문서에 대해서
    for line in document_list:
        doc2vec = None
        count = 0
        for word in line:
            if word in word2vec_model.wv.vocab:
                count += 1
                # 해당 문서에 있는 모든 단어들의 벡터값을 더한다.
                if doc2vec is None:
                    doc2vec = word2vec_model[word]
                else:
                    doc2vec = doc2vec + word2vec_model[word]

        if doc2vec is not None:
            # 단어 벡터를 모두 더한 벡터의 값을 문서 길이로 나눠준다.
            doc2vec = doc2vec / count
            document_embedding_list.append(doc2vec)

    # 각 문서에 대한 문서 벡터 리스트를 리턴
    return document_embedding_list

In [15]:
document_embedding_list = get_document_vectors(books['summary'])
print('문서 벡터의 수 :',len(document_embedding_list))

/var/folders/9d/4103_0g54_s9yr74zb4kcm340000gn/T/ipykernel_94734/3604015244.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  doc2vec = word2vec_model[word]
/var/folders/9d/4103_0g54_s9yr74zb4kcm340000gn/T/ipykernel_94734/3604015244.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  doc2vec = doc2vec + word2vec_model[word]


문서 벡터의 수 : 76118


In [31]:
cosine_similarities = cosine_similarity(document_embedding_list, document_embedding_list)
print('코사인 유사도 매트릭스의 크기 :',cosine_similarities.shape)
print(cosine_similarities)

코사인 유사도 매트릭스의 크기 : (76118, 76118)
[[1.0000004  0.41011983 0.4365611  ... 0.35521767 0.4118845  0.2953902 ]
 [0.41011983 0.9999998  0.59605527 ... 0.6176088  0.6878752  0.56126434]
 [0.4365611  0.59605527 1.         ... 0.48128328 0.5050678  0.45753956]
 ...
 [0.35521767 0.6176088  0.48128328 ... 0.99999994 0.7778654  0.77676535]
 [0.4118845  0.6878752  0.5050678  ... 0.7778654  1.0000002  0.76149   ]
 [0.2953902  0.56126434 0.45753956 ... 0.77676535 0.76149    0.99999976]]


In [21]:
def recommendations(book_id):
    # 책의 제목을 입력하면 해당 제목의 인덱스를 리턴받아 idx에 저장.
    indices = pd.Series(books.index, index = books['id'])
    idx = indices[book_id]

    # 입력된 책과 줄거리(document embedding)가 유사한 책 5개 선정.
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:6]

    # 가장 유사한 책 5권의 인덱스
    book_indices = [i[0] for i in sim_scores]

    # 전체 데이터프레임에서 해당 인덱스의 행만 추출. 5개의 행을 가진다.
    recommend = books.iloc[book_indices].reset_index(drop=True)

    print(recommend)

In [30]:
recommendations(907)

      id                                            summary
0   2943  [그림, 처럼, 다간, 고흐, 의, 마지막, 편지, 장세현, 지은, 이채우, 리반,...
1  51820  [딱, 만, 선택, 라면, 책, 책, 덕, 후, 책, 을, 사랑, 는, 법, 데비,...
2   1745  [유머, 에세이, 장, 김진악, 지은, 범우사, 범우, 문고, 권, 유머, 에세이,...
3  74961  [너, 의, 특별, 달, 지은, 이고은, 그림, 김성미, 달달, 북스, 세상, 에서...
4  11683  [소년, 표류기, 쥘, 베른, 지은, 레옹, 브네, 그림, 김윤진, 옮긴, 비룡소,...
